# Mini project 1, group AM

## Table of content
<!-- [1. Data Preparation](#1.-Data-Preparation)  
[2. Analysis](#2.-Analysis)  
[3. Results](#3.-Results)  -->

[Imports](#imports)

[Part 1](#1)
- [1) Structural preprocessing](#structural-preprocessing)
   - [a) Skull-stripping](#a-skull-stripping)
   - [b) Segmentation](#b-segmentation)
- [2) Functional preprocessing](#functional-preprocessing)
   - [a) Concatenation](#a-concatenation)
   - [b) Motion correction](#b-motion-correction)
   - [c) Co-registration (bonus)](#c-co-registration-(bonus))
   - [d) Gaussian smoothing](#d-gaussian-smoothing)
- [3) Experimental design matrix](#experimental-design-matrix)
- [4) GLM analysis](#glm-analysis)
- [5) Activation maps](#activation-maps)
- [6) Atlas overlay](#atlas-overlay)

[Part 2 : Variant 3](#2)
- [1) K-means clustering](#2-1)
- [2) Selection of a number of clusters](#2-2)
- [3) Pairwise similarity](#2-3)

## Imports

In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt
from fsl.wrappers import mcflirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess
import gc

# Part 1

## 1) Structural preprocessing

### 1)a) Skull-stripping

In [2]:
anatomical_path = op.join("subject101410", "T1w", "T1w.nii.gz") #the original brain
subject_id = '101410'

In [3]:
derivatives = op.join("derivatives", "preprocessed_data", "subject_101410")
deriv_anat = op.join(derivatives, "anat")
deriv_func = op.join(derivatives, "func")

for d in (deriv_anat, deriv_func):
    os.makedirs(d, exist_ok=True)

print("Derivatives will be written to:", os.path.abspath(derivatives))

Derivatives will be written to: /data/NX-NSSP/derivatives/preprocessed_data/subject_101410


In [4]:
#bids_root   = "/data/NX-NSSP"  
#preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
#deriv_root = op.join(bids_root, 'derivatives')

In [5]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

08:52:04: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:52:04: Debug: Adding duplicate animation handler for '1' type
08:52:04: Debug: Adding duplicate animation handler for '2' type
08:52:04: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:52:04: Debug: Adding duplicate animation handler for '1' type
08:52:04: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:3627): Gtk-CRITICAL **: 08:52:04.426: gtk_window_resize: assertion 'height > 0' failed


In [6]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anatomical_path)


(ipykernel_launcher.py:3627): Gdk-WARNING **: 08:52:08.432: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [ ]:
def get_skull_stripped_anatomical(deriv_anat, anatomical_path, subject_id, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    # We perform here skull stripping (you'll learn more about it next week!).
    # For now all you need to do is that we remove the bones and flesh from the MRI to get the brain!
    betted_brain_path = op.join(deriv_anat, 'sub-{}_T1w'.format(subject_id))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(deriv_anat, 'sub-101410_T1w_mask')
get_skull_stripped_anatomical(deriv_anat, anatomical_path, subject_id, robust=True)

In [ ]:
fsleyesDisplay.load(resulting_mask_path)

In [ ]:
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    
betted_brain_path = op.join(deriv_anat, 'sub-101410_T1w.nii.gz') # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(deriv_anat, 'sub-101410_T1w_mask.nii.gz') # The mask to use

apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

### 1)b) Segmentation

In [ ]:
bet_path = op.join(deriv_anat, 'sub-101410_T1w')
fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(deriv_anat, '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(deriv_anat, 'sub-101410_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

pve_0 in red : CSF \
pve_1 in green : grey matter \
pve_2 in blue : white matter

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## 2) Functional preprocessing

### 2)a) Concatenation

In [9]:
# Code to verify folder size
total_size = sum(os.path.getsize(op.join(deriv_func, f)) for f in os.listdir(deriv_func))
print(f"Remaining size in derivatives folder: {total_size / (1024**2):.2f} MB")


Remaining size in derivatives folder: 953.60 MB


In [10]:
# Create functional paths
functional_path_LR = op.join('subject101410', 'fMRI', 'tfMRI_MOTOR_LR', 'tfMRI_MOTOR_LR.nii') 
functional_path_RL = op.join('subject101410', 'fMRI', 'tfMRI_MOTOR_RL', 'tfMRI_MOTOR_RL.nii') 

print("Functional paths have been created")

Functional paths have been created


In [11]:
###########################################
# Variance Normalisation - Run LR
###########################################
print("Normalizing variance for LR run")

# Calculate the variance for LR
variance_LR = subprocess.check_output(['fslstats', functional_path_LR, '-V']).decode().split()[1]
print(f"Global variance for LR: {variance_LR}")

# Compute the scaling factor (1 / sqrt(variance))
scale_LR = 1 / (float(variance_LR) ** 0.5)
print(f"Scaling factor for LR: {scale_LR}")

# Apply scaling to normalize the variance
scaled_LR_path = op.join(deriv_func, 'tfMRI_MOTOR_LR_varscaled.nii.gz')
subprocess.run(['fslmaths', functional_path_LR, '-mul', str(scale_LR), scaled_LR_path], check=True)
print(f"Saved normalized LR run: {scaled_LR_path}\n")

# Delete intermediary variables to free memory
del variance_LR, scale_LR
gc.collect()

###########################################
# Variance Normalisation - Run RL
###########################################
print("Normalizing variance for RL run")

# Calculate the variance for RL
variance_RL = subprocess.check_output(['fslstats', functional_path_RL, '-V']).decode().split()[1]
print(f"Global variance for RL: {variance_RL}")

# Compute the scaling factor for RL (1 / sqrt(variance))
scale_RL = 1 / (float(variance_RL) ** 0.5)
print(f"Scaling factor for RL: {scale_RL}")

# Apply scaling to normalize the variance
scaled_RL_path = op.join(deriv_func, 'tfMRI_MOTOR_RL_varscaled.nii.gz')
subprocess.run(['fslmaths', functional_path_RL, '-mul', str(scale_RL), scaled_RL_path], check=True)
print(f"Saved normalized RL run: {scaled_RL_path}\n")

# Delete intermediary variables to free memory
del variance_RL, scale_RL
gc.collect()

###########################################
# Concatenate the two runs
###########################################
print("Concatenating normalized runs...")

concat_path = op.join(deriv_func, f"sub-{subject_id}_motor_concat_varscaled.nii.gz")
subprocess.run(['fslmerge', '-t', concat_path, scaled_LR_path, scaled_RL_path], check=True)

print(f"Done! Final concatenated file: {concat_path}")

Normalizing variance for LR run
Global variance for LR: 519161344.000000
Scaling factor for LR: 4.388830677732065e-05
Saved normalized LR run: derivatives/preprocessed_data/subject_101410/func/tfMRI_MOTOR_LR_varscaled.nii.gz

Normalizing variance for RL run
Global variance for RL: 519202016.000000
Scaling factor for RL: 4.3886587735293315e-05
Saved normalized RL run: derivatives/preprocessed_data/subject_101410/func/tfMRI_MOTOR_RL_varscaled.nii.gz

Concatenating normalized runs...
Done! Final concatenated file: derivatives/preprocessed_data/subject_101410/func/sub-101410_motor_concat_varscaled.nii.gz


In [ ]:
# Delete unnecessary files (RL and LR before concatanation)
del scaled_LR_path, scaled_RL_path
gc.collect()

### 2)b) Motion correction

In [ ]:
# Correct the motion
print("Starting the motion correction")
path_motioncorrected = op.join(deriv_func, 'tfMRI_MOTOR_motioncorrected') # path to the motion corrected file
mcflirt(infile=concat_path, o=path_motioncorrected, plots=True, report=True, dof=6, mats=True)
print("Motion corrected")

In [ ]:
# Display result
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_path)
fsleyesDisplay.load(path_motioncorrected)
print("Motion corrected done")

In [ ]:
import matplotlib.pyplot as plt

mc_par_path = op.join(deriv_func, 'tfMRI_MOTOR_motioncorrected.par')

def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

mot_params = load_mot_params_fsl_6_dof(mc_par_path)
mot_params

def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rotation_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    
    convert_rot_to_arc = rotation_params*90                                    # transform from angle to arc of circle for a radius of 50mm, normal used values are 50 or 90 mm
    translation_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = convert_rot_to_arc.abs().sum(axis=1) + translation_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()
threshold = np.quantile(fd,0.75) + 1.5*(np.quantile(fd,0.75) - np.quantile(fd,0.25))

#%matplotlib inline
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

print(f"Indices of frames of motion above threshold: {(np.where(fd > threshold)[0] + 1).tolist()}")

In [ ]:
# Run fsl_motion_outliers to detect bad frames
motion_no_outliers = f"{path_motioncorrected}.nii.gz"
motion_outliers_confounds = op.join(deriv_func, 'motion_outliers_confounds.txt')
motion_outliers_plot = op.join(deriv_func, 'motion_outliers_plot.png')
motion_outliers_summary = op.join(deriv_func, 'motion_outliers_summary.txt')

# Run fsl_motion_outliers command
print("Running fsl_motion_outliers")
subprocess.run(f"fsl_motion_outliers -i {motion_no_outliers} -o {motion_outliers_confounds} -p {motion_outliers_plot} -s {motion_outliers_summary} --fd --thresh=0.5", shell=True)
print("Motion outliers detection completed")

In [ ]:
# # Cell to run if save is not possible anymore

import shutil
import os

folder_to_clear = "derivatives/preprocessed_data/subject_101410"
if os.path.exists(folder_to_clear):
     shutil.rmtree(folder_to_clear)
     print("Dossier supprimé pour libérer de l'espace.")

# In order to determine how many in how many images the subject moves more than what is tolerated, we can run the cell below

### 2)c) Co-registration (bonus)

In [7]:
import nibabel as nib

In [8]:
#redefine the path if you have not run all the cells
betted_brain_path = op.join(deriv_anat, 'sub-{}_T1w'.format(subject_id))
concat_path = os.path.join(deriv_func, f"sub-{subject_id}_motor_concat_varscaled.nii.gz")

#set reference = anatomical image and source fonctional image
reference= betted_brain_path
source= nib.load(concat_path)
data_source = source.get_fdata()
print("Dimensions :", data_source.shape)

#select on volume arbitrary 
volume_index = 10  
volume_source = data_source[..., volume_index]
print(volume_source.shape) 

#convert in tuple
spacing = tuple(source.header.get_zooms()[:3])
origin = tuple(source.affine[:3, 3])
#Apply ANTS
moving_image = ants.from_numpy(volume_source, origin=origin, spacing=spacing)
fixed_image = ants.image_read(reference + '.nii.gz')

transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform = 'SyN' )
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

# Save the image to disk
resultAnts = op.join(deriv_anat,'sub-{}_T1w_mni_SyN.nii.gz'.format(subject_id))
ants.image_write(warpedImage, resultAnts)



Dimensions : (91, 109, 91, 568)
(91, 109, 91)


In [9]:
#Visualize the results functional overlay on anatomical
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(resultAnts)

### 2)d) Gaussian smoothing

In [10]:

output_path = concat_path
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(output_path, 6/2.3548, output_path)
subprocess.run(['fslmaths',output_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])
results_smoothing = '{}_smoothed-6mm.nii.gz'.format(output_path)


In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(output_path) 
fsleyesDisplay.load(results_smoothing)

## 3) Experimental design matrix

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.datasets import fetch_atlas_aal
from nilearn.plotting import plot_stat_map
from IPython.display import display
from nilearn.image import mean_img
import nibabel as nib

In [ ]:
fMRI_path = '/data/subject101410/fMRI'

func_path_LR = op.join(fMRI_path, 'tfMRI_MOTOR_LR') 
func_path_RL = op.join(fMRI_path, 'tfMRI_MOTOR_RL') 

events_LR = pd.read_csv(op.join(func_path_LR, 'events_LR.csv'), sep=',')
events_RL = pd.read_csv(op.join(func_path_RL, 'events_RL.csv'), sep=',')

print("Events Table — LR Run")
pd.set_option('display.float_format', '{:,.2f}'.format)
display(events_LR.head(10))

print("Events Table — RL Run")
pd.set_option('display.float_format', '{:,.2f}'.format)
display(events_RL.head(10))


In [ ]:
img = nib.load(func_img)
n_scans = img.shape[3]           # number of volumes
TR = img.header.get_zooms()[3]   # repetition time
frame_times = np.arange(n_scans) * TR  # time vector for each volume

design_matrix = make_first_level_design_matrix(
    frame_times,
    events_RL,
    hrf_model='spm',
    drift_model='cosine',
    high_pass=0.01)

print('Design Matrix ')
plot_design_matrix(fmri_glm_.design_matrices_[0])
plt.show()

## 4) GLM analysis

In [ ]:
GLM_fMRI = FirstLevelModel(t_r=0.72,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01)

# Fit the model to our design and data
fmri_glm = GLM_fMRI.fit(fmri_img, events_LR)

# Get regressors
regressors = fmri_glm.design_matrices_[0].columns.tolist()
print(regressors)
regressors = fmri_glm.design_matrices_[0].columns.tolist()
print(regressors)

# Compute and plot t-map for each regressor and run
for reg in regressors:
    t_map = fmri_glm.compute_contrast(reg, output_type='stat')
    plotting.plot_stat_map(
        t_map,
        threshold=3.0,
        display_mode='ortho',
        title=f'Statistical map: {reg}'
    )

# Comments on t-map

## 5) Activation maps

In [ ]:
contrast_vector_hand_vs_foot = [1, 1, -1, -1, 0, 0] # depend on the regressor list, might be changed

activation_map = fmri_glm.compute_contrast(contrast_vector_hand_vs_foot,
                                  output_type='effect_size')
# activation map
plotting.plot_stat_map(
    activation_map
    threshold=0.5,          # might need to be adjusted as needed
    display_mode='ortho',
    cut_coords=(0, 0, 0),   # might need to be adjusted as needed
    title='Hand > Foot Activation Map'
)
plt.show()

#nib.save(activation_map, 'activation_map_hand_vs_foot.nii.gz')

## 6) Atlas overlay

In [ ]:
from nilearn import plotting
atlas = fetch_atlas_aal()

plotting.plot_stat_map(
    activation_map,
    threshold=2.5,
    display_mode='ortho',
    title='Activation Map (Hand > Foot) with AAL Overlay',
    cut_coords=(0, 0, 0)
)

plotting.plot_roi(
    atlas.maps,
    bg_img=activation_map,
    alpha=0.3,
    display_mode='ortho',
    cut_coords=(0, 0, 0)
)

# Part 2 : Variant 3
## 1) K-means clustering

In [ ]:
# fMRI_path = '/data/NX-NSSP/subject101410/fMRI'
# func_path_LR = op.join(fMRI_path, 'tfMRI_MOTOR_LR') 
# func_path_RL = op.join(fMRI_path, 'tfMRI_MOTOR_RL') 

img = nib.load('derivatives/preprocessed_data/subject_101410/func/sub-101410_motor_concat_varscaled.nii.gz')
#Loading data
# img = nib.load(op.join(func_path_LR,'tfMRI_MOTOR_LR.nii'))
affine = img.affine
data = np.asanyarray(img.dataobj)
print(data.shape)


In [ ]:
#Preprocessing

# Make variables:
# 'vol_shape' for shape of volumes
# 'n_vols' for number of volumes
# YOUR CODE HERE
vol_shape = data.shape[:3]
n_vols    = data.shape[3]

# YOUR CODE HERE
# Note: In our case the background is encoded as 0 
# you can consider that the first volume's background 
# voxels are the same as all following volumes

slice_non_background = data[..., 0] != 0
# Vectorize : Taking only non-zero voxels into a vector 
# (NOTE: that the order is important)
samples = data[slice_non_background, :]

# Calculate the mean across columns
spatial_means = samples.mean(axis=1, keepdims=True)   # shape: (n_vols, 1)

# Row means copied n_vols times so that we substract for each timepoint the spatial mean
row_means = np.repeat(spatial_means, samples.shape[1], axis=1) # shape: (n_vols, n_voxels)

# Subtract the means for each row, put the result into X
X_kmeans = samples - row_means

# Verify that the spatial mean behaves as expected after substraction
print("Mean after centering (should be ~0):", np.mean(X_kmeans, axis=1))


In [ ]:
print(samples.shape)

In [ ]:
X_kmeans.shape

In [ ]:
from sklearn.cluster import KMeans

max_nb_clusters = 10
avg_dist_samples = []
for nb_cluster in range(1, max_nb_clusters):
    kmeans = KMeans(n_clusters=nb_cluster, random_state=0, n_init="auto").fit(X_kmeans.T)
    avg_dist_samples.append(kmeans.inertia_)

## 2) Selection of a number of clusters

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, figsize=(7,5))

ax.plot(np.arange(1, len(avg_dist_samples)+1), avg_dist_samples, label='explained variance ratios', c='k')

ax.set_xlabel('Clusters #', size=15)
ax.legend(prop={'size':15})
ax.tick_params(axis='both', which='major', labelsize=15)

In [ ]:
# YOUR CODE HERE

nb_clusters = 2 # The number of clusters you want
kmeans = KMeans(n_clusters=nb_cluster, random_state=0, n_init="auto").fit(X_kmeans.T)

kmeans_clusters = [] # List of spatial components (you should have in the list volumes)
for i in range(nb_clusters):
    centroid_3d = np.zeros(vol_shape)         # full 3D shape
    centroid_3d[slice_non_background] = kmeans.cluster_centers_[i]
    kmeans_clusters.append(centroid_3d)

In [ ]:
from nilearn.plotting import plot_stat_map
from nilearn.image import mean_img
mean_img_ = mean_img(img)
visual_idx = 0
plot_stat_map(nib.Nifti1Image(kmeans_clusters[visual_idx], affine), bg_img=mean_img_, threshold=0,
               cut_coords=[49,37,00], black_bg=True,
              title=f'K-means Cluster {visual_idx}')

plt.show()

In [ ]:
mean_img_ = mean_img(img)
zmax = vol_shape[2] - 1
cut_coords = list(np.linspace(0, zmax, 9, dtype=int))  # 9 axial slices

for i, centroid in enumerate(kmeans_clusters):
    print(f"Cluster {i+1}")
    plot_stat_map(
        nib.Nifti1Image(centroid, affine),
        bg_img=mean_img_,
        display_mode='z',
        cut_coords=cut_coords,
        threshold=0,
        colorbar=True,
        black_bg=True,
        title=f"K-means Cluster {i+1}"
    )

## 3) Pairwise similarity

In [ ]:
nb_clusters = 5
kmeans = KMeans(n_clusters=nb_clusters, random_state=0, n_init="auto").fit(X_kmeans)


In [ ]:
from scipy.stats import pearsonr

# cluster_centers_: shape (5, n_voxels)
C = kmeans.cluster_centers_

m = 5
sim = np.zeros((m, m))

for i in range(m):
    for j in range(m):
        sim[i, j] = pearsonr(C[i], C[j])[0]  # correlation coefficient

print("Similarity matrix (Pearson):")
print(np.round(sim, 3))


# END